In [1]:
import numpy as np 
import pandas as pd 
import scipy.optimize 

In [2]:
p=2
theta_bounds = [(-0.5, 1.5)]*(1+p+1)
psi_max = np.inf 

In [3]:
def psi(design=None,theta=None):
    rho=0.3 #Cross-Correlation Coefficient
    
    X1 = np.array([[1, 1, 1, 0],[1, -1, 1, 1]]) 
    X2 = np.array([[1, 1, 1, 0],[1, -1, -1, 1]]) 
    X3 = np.array([[1, 1, -1, 0],[1, -1, 1, -1]]) 
    X4 = np.array([[1, 1, -1, 0],[1, -1, -1, -1]]) 
    
#     print(X1.shape,theta.shape)
#     print(theta)
    
    n1=X1@theta.T
    n2=X2@theta.T
    n3=X3@theta.T
    n4=X4@theta.T
    
    u1=np.exp(n1)/(1+np.exp(n1))
    u2=np.exp(n2)/(1+np.exp(n2))
    u3=np.exp(n3)/(1+np.exp(n3))
    u4=np.exp(n4)/(1+np.exp(n4))
    
    A1=np.diag(u1*(1-u1))
    A2=np.diag(u2*(1-u2))
    A3=np.diag(u3*(1-u3))
    A4=np.diag(u4*(1-u4))
#     print("A1.shape=",A1.shape)
    
    R= rho*np.ones((p,p))+(1-rho)*np.eye(p)
    R_inv = np.linalg.inv(R)
    
#     (4, 2) (2,2) (2, 2) (2,2) (2, 4)
#     print(X1.T.shape,np.sqrt(A1).shape,R_inv.shape,np.sqrt(A1).shape,X1.shape)

    asymptotic_information_matrix = design[0]*X1.T@np.sqrt(A1)@R_inv@np.sqrt(A1)@X1 +design[1]*X2.T@np.sqrt(A2)@R_inv@np.sqrt(A2)@X2 +design[2]*X3.T@np.sqrt(A3)@R_inv@np.sqrt(A3)@X3 +design[3]*X4.T@np.sqrt(A4)@R_inv@np.sqrt(A4)@X4
    
    asymptotic_variance = np.linalg.inv(asymptotic_information_matrix)
    C=asymptotic_variance[2,2]
    return C 

    
psi([0.25, 0.25, 0.25, 0.25],np.array([1.0, 1.0, 1.0, 1.0]) )

4.490771750442258

In [4]:
def B(design, theta_vals, prob_vals):
    s1 = sum([prob_vals[i] * psi(design,theta_vals[i]) for i in range(len(theta_vals))])
    return s1 

B([0.25, 0.25, 0.25, 0.25], [np.array([1.0, 1.0, 1.0, 1.0])], [1.0]  )

4.490771750442258

In [5]:
def get_ootad(theta_vals, prob_vals):  
    B_pi = lambda des: B(des,theta_vals,prob_vals) 
    
    bd= [(1e-12,1-1e-12)]*(1+p+1)
    def cnst(x):
        if abs(1-sum(x))<1e-5:
            return 0 
        else:
            return -1    
    opt_des = None; min_B=np.inf;
    num_trials = 100 
    for trial in range(num_trials):  
        
        sample = np.random.random_sample(size=4)
        des0 = sample/np.sum(sample)        
        res = scipy.optimize.minimize(B_pi,des0,bounds=bd,constraints={'fun':cnst,'type':'eq'})
        if abs(sum(res.x)-1)>0.01:
            print("?")
        
        if res.fun<min_B:
            min_B=res.fun
            opt_des=res.x
    
    return opt_des
        
        
get_ootad([np.array([1.0, 1.0, 1.0, 1.0])],[1.0]) 

array([0.35667055, 0.28776547, 0.15123807, 0.20432591])

In [6]:
def get_least_fav_theta(des):
    global psi_max
    
    psi_theta = lambda theta: -1*psi(des,theta)
    bd = theta_bounds
    
    opt_theta=None; 
    psi_max=-np.inf;
    num_trials = 100 
    for trial in range(num_trials):
        theta0 = np.array([np.random.uniform(low=theta_bounds[i][0],high=theta_bounds[i][1]) for i in range(1+p+1)])
        res = scipy.optimize.minimize(psi_theta,theta0,bounds=theta_bounds)
#         print(res)
        if -1*res.fun>psi_max:
            psi_max = -1*res.fun
            opt_theta=res.x 
    
#     print("psi_max=",psi_max)
    
    return opt_theta
        
        
get_least_fav_theta([0.25, 0.25, 0.25, 0.25]) 

array([1.5, 1.5, 1.5, 1.5])

In [7]:
l = 1
B_pi_l = -np.inf
steps = 5
H = np.linspace(0.0,1.0,steps)
print(H)

#Initial Prior
theta_vals_l = [np.ones((1+p+1))]
prob_vals_l = np.array([1.0])

# psi_l = get_ootad()
print(theta_vals_l)
print(prob_vals_l)

des_l = get_ootad(theta_vals_l,prob_vals_l)
B_pi_l = B(des_l,theta_vals_l,prob_vals_l)

print(B_pi_l)

[0.   0.25 0.5  0.75 1.  ]
[array([1., 1., 1., 1.])]
[1.]
4.275068941945748


In [8]:
while(True):
    theta_l = get_least_fav_theta(des_l)
#     print(type(theta_l))
    T = len(H) 
    B_pi_l1 = -np.inf 
    theta_vals_l1, prob_vals_l1, des_l1 = None, None, None;
    
    while(B_pi_l1<B_pi_l):        
        
        for t in range(T):
            #generating new prior 
            theta_vals_l_t = theta_vals_l + [ theta_l ]
            prob_vals_l_t = [(1-H[t])*prob_vals_l[i] for i in range(len(prob_vals_l))] + [H[t]]
            des_l_t = get_ootad(theta_vals_l_t,prob_vals_l_t)
            B_pi_l_t = B(des_l_t,theta_vals_l_t,prob_vals_l_t) 

            if B_pi_l_t>B_pi_l1:
                theta_vals_l1 = theta_vals_l_t
                prob_vals_l1 = prob_vals_l_t
                des_l1 = des_l_t 
                B_pi_l1 = B_pi_l_t 
                
        if (B_pi_l1<B_pi_l):            
            steps = steps*2
            H = np.linspace(0.0,1.0,steps)
        
    
    #Setting l as l+1
    theta_vals_l = theta_vals_l1
    prob_vals_l = prob_vals_l1
    des_l = des_l1 
    B_pi_l = B_pi_l1 
    l+=1;
    
    print("Iter",l-1,"psi_max=",psi_max,"\tB_pi_l=",B_pi_l)
    
    if psi_max<=B_pi_l:
#         print("theta_vals_l=\n",theta_vals_l) 
#         print("prob_vals_l=\n",prob_vals_l) 
        for i in range(len(prob_vals_l)):
            print(theta_vals_l[i],"with prob",prob_vals_l[i])
        break

Iter 1 psi_max= 9.672805963166335 	B_pi_l= 9.108738439375172
Iter 2 psi_max= 9.492856499851834 	B_pi_l= 9.228267196949247
Iter 3 psi_max= 9.428045638209202 	B_pi_l= 9.426842640220121
Iter 4 psi_max= 9.482235445216215 	B_pi_l= 9.438603326894928
Iter 5 psi_max= 9.472381982447581 	B_pi_l= 9.606203985035677
[1. 1. 1. 1.] with prob 0.0
[1.5 1.5 1.5 1.5] with prob 0.13444894220358966
[1.5        1.07731599 1.5        1.5       ] with prob 0.03585305125429057
[1.5 1.5 1.5 1.5] with prob 0.3689876524920737
[1.5 1.5 1.5 1.5] with prob 0.0023281735107729563
[1.5 1.5 1.5 1.5] with prob 0.4583821805392732


In [9]:
def see_pmf(theta_vals,prob_vals):
    theta_vals = tuple(map(tuple, theta_vals))
    d = pd.DataFrame(list(zip(theta_vals,prob_vals)),columns=["Theta", "Prob"])
#     print("Original\n",d)
    d = d.groupby(["Theta"]).aggregate(sum)
    print(d)

# print(np.array(theta_vals_l))
# print(prob_vals_l)
see_pmf(theta_vals_l,prob_vals_l)

                                         Prob
Theta                                        
(1.0, 1.0, 1.0, 1.0)                 0.000000
(1.5, 1.0773159896639666, 1.5, 1.5)  0.035853
(1.5, 1.5, 1.5, 1.5)                 0.964147
